In [ ]:
library(tidyverse)
library(repr)
library(gridExtra)

Select neccesary data

In [ ]:
history_data <- read_csv("data/history.csv")
statement_raw_data <- read_csv("data/statement.csv") %>%
    rbind(history_data)

#write_csv(statement_raw_data, "data/history.csv", append = FALSE)
#TODO: REMOVE DUPLICATE ROWS

visa_data <- statement_raw_data %>%
    filter(`Account Type`=="Visa") %>%
    select("Transaction Date", "Description 1", "CAD$")%>%
    rename("Description 2"=`Description 1`)

chequing_data <- statement_raw_data %>%
    filter(`Account Type` == "Chequing") %>%
    select("Transaction Date", "Description 2", "CAD$")

statement_data <- rbind(chequing_data, visa_data) %>%
    rename("Transaction"="CAD$")

head(statement_data, n=10)

Categorize transactions

In [ ]:
lookup_data <- read_csv("data/lookup_table.csv")
head(lookup_data)

Merge transaction type with remaining data and remove vendor name

In [ ]:
merged_data <- merge(statement_data, lookup_data)%>%
    select(-"Description 2")
head(merged_data, n=10)

Minor clean up

In [ ]:
savings_filter <- filter(merged_data, type=="Savings")%>%
    mutate(Transaction= abs(as.numeric(Transaction)))
cleaned_data <- filter(merged_data, type!="Savings")%>%
    rbind(savings_filter)%>%
    mutate(Transaction= as.numeric(Transaction))
head(cleaned_data, n=10)

Separate Transaction date into multiple columns

In [ ]:
date_no_delimiter <- separate(cleaned_data,
                             col = "Transaction Date",
                             into = c("Month", "Day", "Year"),
                             sep = "/")
date_sorted <- arrange(date_no_delimiter, Year, Month, Day)
head(date_sorted)

Group by month

In [ ]:
monthly_summary <- aggregate(date_no_delimiter$Transaction, by=list(Type=date_no_delimiter$type,
                                                                Month=date_no_delimiter$Month,
                                                                Year=date_no_delimiter$Year),
                            FUN=sum)
head(monthly_summary)

Visualizations

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15)
usage_chart <- ggplot(monthly_summary, aes(x=Month, y=x, fill=factor(Type, levels=
                                                                          c("Allowance",
                                                                            "Deposit",                                                                
                                                                            "Housing", 
                                                                            "Subscriptions",
                                                                            "Withdrawal",
                                                                            "Groceries", 
                                                                            "Personal", 
                                                                            "Restaurant",
                                                                            "Ticketing",
                                                                            "Clothing",
                                                                            "Savings", 
                                                                            "Misc")))) +
    geom_bar(stat = "identity", position = position_stack(reverse = TRUE)) + 
    labs(x="Month", y="Dollars Spent", fill="Transaction Type")+
    scale_y_reverse()+ 
    scale_fill_brewer(palette='Set3')+
    theme(text = element_text(size = 20))
usage_chart

In [ ]:
final_summary <- monthly_summary %>%
    mutate(x=abs(x))%>%
    pivot_wider(names_from = Type, values_from = x)
final_summary[is.na(final_summary)] <- 0
final_summary[,c(1,2,3,4,8,5,6,7,10,9,12,13,15,11)]

In [ ]:
total_summary <- final_summary%>%
    mutate(`Total Inflow`=Allowance+Deposit+Savings)%>%
    mutate(`Total Outflow`=Withdrawal+Groceries+Personal+Restaurant+Subscriptions+Misc)%>%
    mutate(`Total Outflow (+ Housing)` = `Total Outflow` + Housing)%>%
    mutate(`Carryover Balance` = `Total Inflow`-`Total Outflow (+ Housing)`)%>%
    select(Month,`Total Inflow`,`Total Outflow`,`Total Outflow (+ Housing)`,`Carryover Balance`)
total_summary